# Example 1: 秒単位インターバル分析

このノートブックは、DuckDB × MCP × LLM アーキテクチャを使用した秒単位のペース変動分析の例を示します。

## アーキテクチャのポイント

1. **LLM**: データ要求の計画と結果の解釈のみを行う
2. **MCP Server**: `export()` でデータをParquetとしてエクスポート（ハンドルのみ返却）
3. **Python**: 実際のデータ処理（リサンプリング、ローリング平均、グラフ生成）
4. **Output Validation**: 要約データのみをLLMに返却（生データは返さない）

## Before: 従来の方法（コンテキスト膨張）

```python
# 従来: 生データを直接読み込み（1000-2000行 × 26カラム = 数十KB）
data = mcp__garmin_db__get_time_range_detail(
    activity_id=12345,
    start_time_s=300,
    end_time_s=600,
    statistics_only=False  # 全データ返却
)
# 返却サイズ: 約50KB（トークン: 約12,500）
```

## After: 新アーキテクチャ（トークン削減）

```python
# ステップ1: データをエクスポート（ハンドルのみ返却）
export_result = mcp__garmin_db__export(
    query="""
        SELECT timestamp, pace, heart_rate, cadence
        FROM time_series_metrics
        WHERE activity_id = 12345
        AND timestamp BETWEEN 300 AND 600
    """,
    format="parquet"
)
# 返却サイズ: 約100バイト（トークン: 約25）
# トークン削減: 99.8%

# ステップ2: Pythonでデータ処理
df = safe_load_export(export_result['handle'])
df_resampled = df.resample('1S').mean()
rolling_avg = df_resampled['pace'].rolling(10).mean()

# ステップ3: 要約データのみ返却
summary = safe_json_output({
    'avg_pace': df['pace'].mean(),
    'std_pace': df['pace'].std(),
    'max_deviation': (df['pace'] - rolling_avg).abs().max()
})
# 返却サイズ: 約200バイト（トークン: 約50）
```

## セットアップ

In [ ]:
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import polars as pl

# プロジェクトルートをPATHに追加
project_root = Path().absolute().parent.parent
sys.path.insert(0, str(project_root))

from tools.utils.llm_safe_data import (
    safe_json_output,
    safe_load_export,
    safe_summary_table,
)

## モックデータの作成

実際の環境では、MCP Server の `export()` 関数がデータを Parquet ファイルとしてエクスポートします。
ここでは、その結果をシミュレートします。

In [ ]:
# モックデータ: 5分間（300秒）の秒単位データ
np.random.seed(42)
n_seconds = 300

# ペース: 4:30/km ± ランダム変動 + 給水で急減速（7:30付近）
base_pace = 270  # 4:30/km = 270秒/km
pace = base_pace + np.random.normal(0, 5, n_seconds)
pace[150:160] += 20  # 2:30-2:40で給水による急減速

# 心拍: 150bpm ± ランダム変動
heart_rate = 150 + np.random.normal(0, 3, n_seconds)

# ケイデンス: 180spm ± ランダム変動
cadence = 180 + np.random.normal(0, 2, n_seconds)

# タイムスタンプ（開始から300-600秒）
timestamp = np.arange(300, 600)

df = pl.DataFrame({
    'timestamp': timestamp,
    'pace': pace,
    'heart_rate': heart_rate,
    'cadence': cadence
})

# Parquetとして保存（実際のexport()の動作をシミュレート）
export_path = Path('/tmp/export_example1.parquet')
df.write_parquet(export_path)

print(f"✓ モックデータをエクスポート: {export_path}")
print(f"  行数: {len(df)}")
print(f"  サイズ: {export_path.stat().st_size / 1024:.1f} KB")

## ステップ1: MCP Server の `export()` 呼び出し（シミュレート）

実際の環境では、以下のようにMCP関数を呼び出します：

```python
export_result = mcp__garmin_db__export(
    query="""
        SELECT timestamp, pace, heart_rate, cadence
        FROM time_series_metrics
        WHERE activity_id = 12345
        AND timestamp BETWEEN 300 AND 600
    """,
    format="parquet"
)
```

In [ ]:
# MCP Server のレスポンスをシミュレート（ハンドルのみ、データなし）
export_result = {
    'handle': str(export_path),
    'rows': len(df),
    'size_mb': export_path.stat().st_size / (1024 * 1024),
    'columns': ['timestamp', 'pace', 'heart_rate', 'cadence']
}

print("MCP Server レスポンス（ハンドルのみ）:")
print(safe_json_output(export_result))
print("\nトークンコスト: 約25トークン（生データなし）")

## ステップ2: Python でデータ処理

ハンドルを使用してデータをロードし、リサンプリング・ローリング平均・グラフ生成を行います。

In [ ]:
# データをロード（safe_load_export でサイズ制限を適用）
df_loaded = safe_load_export(export_result['handle'], max_rows=10000)

print(f"✓ データをロード: {len(df_loaded)} 行")
print("\nデータプレビュー（先頭5行）:")
print(safe_summary_table(df_loaded, max_rows=5))

In [ ]:
# ローリング平均を計算（10秒窓）
df_with_rolling = df_loaded.with_columns([
    pl.col('pace').rolling_mean(window_size=10).alias('pace_rolling_10s')
])

# ペースの偏差を計算
df_with_rolling = df_with_rolling.with_columns([
    (pl.col('pace') - pl.col('pace_rolling_10s')).abs().alias('pace_deviation')
])

print("✓ ローリング平均を計算")

In [ ]:
# グラフ生成
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# ペースの時系列プロット
ax1.plot(df_with_rolling['timestamp'], df_with_rolling['pace'],
         alpha=0.5, label='生ペース', linewidth=1)
ax1.plot(df_with_rolling['timestamp'], df_with_rolling['pace_rolling_10s'],
         color='red', label='ローリング平均（10秒）', linewidth=2)
ax1.set_xlabel('タイムスタンプ（秒）')
ax1.set_ylabel('ペース（秒/km）')
ax1.set_title('5:00-10:00のペース変動')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.invert_yaxis()  # ペースは小さいほど速い

# ペース偏差のプロット
ax2.plot(df_with_rolling['timestamp'], df_with_rolling['pace_deviation'],
         color='orange', linewidth=1)
ax2.axhline(y=df_with_rolling['pace_deviation'].mean(),
            color='red', linestyle='--', label='平均偏差')
ax2.set_xlabel('タイムスタンプ（秒）')
ax2.set_ylabel('ペース偏差（秒/km）')
ax2.set_title('ローリング平均からのペース偏差')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plot_path = Path('/tmp/interval_analysis_plot.png')
plt.savefig(plot_path, dpi=100, bbox_inches='tight')
plt.show()

print(f"✓ グラフを保存: {plot_path}")

## ステップ3: 要約データのみをLLMに返却

生データではなく、統計サマリーのみを返却します（1KB制限）。

In [ ]:
# 統計サマリーを計算
summary = {
    'time_range': {
        'start': int(df_with_rolling['timestamp'].min()),
        'end': int(df_with_rolling['timestamp'].max()),
        'duration_minutes': (df_with_rolling['timestamp'].max() - df_with_rolling['timestamp'].min()) / 60
    },
    'pace_stats': {
        'avg_sec_per_km': float(df_with_rolling['pace'].mean()),
        'std_sec_per_km': float(df_with_rolling['pace'].std()),
        'min_sec_per_km': float(df_with_rolling['pace'].min()),
        'max_sec_per_km': float(df_with_rolling['pace'].max())
    },
    'deviation_stats': {
        'avg_deviation': float(df_with_rolling['pace_deviation'].mean()),
        'max_deviation': float(df_with_rolling['pace_deviation'].max()),
        'max_deviation_timestamp': int(df_with_rolling.filter(
            pl.col('pace_deviation') == pl.col('pace_deviation').max()
        )['timestamp'][0])
    },
    'plot_path': str(plot_path)
}

# 要約JSONを出力（1KB制限）
summary_json = safe_json_output(summary)
print("要約データ（LLMに返却）:")
print(summary_json)
print(f"\nJSON サイズ: {len(summary_json.encode('utf-8'))} バイト")
print("トークンコスト: 約50トークン")

## ステップ4: LLMによる結果解釈（シミュレート）

LLMは要約データとグラフのみを受け取り、自然言語で解釈します。

In [ ]:
# LLMの解釈例（実際にはLLMが生成）
interpretation = f"""
**5:00-10:00の区間分析結果:**

- 平均ペース: {summary['pace_stats']['avg_sec_per_km']:.1f}秒/km（約{summary['pace_stats']['avg_sec_per_km']//60:.0f}:{summary['pace_stats']['avg_sec_per_km']%60:.0f}/km）
- 標準偏差: {summary['pace_stats']['std_sec_per_km']:.1f}秒/km
- 最大偏差: {summary['deviation_stats']['max_deviation']:.1f}秒/km（タイムスタンプ {summary['deviation_stats']['max_deviation_timestamp']}秒付近）

**観察:**
- {summary['deviation_stats']['max_deviation_timestamp']}秒付近（約{summary['deviation_stats']['max_deviation_timestamp']//60:.0f}:{summary['deviation_stats']['max_deviation_timestamp']%60:02.0f}）でペースが急減速しています
- この急減速は給水ポイントの影響と推測されます
- その他の区間では比較的安定したペースを維持しています（標準偏差 {summary['pace_stats']['std_sec_per_km']:.1f}秒）
"""

print(interpretation)

## トークンコスト比較

### Before（従来の方法）:
- 生データ返却: 300行 × 4カラム × 平均10バイト = 約12KB
- トークンコスト: 約3,000トークン

### After（新アーキテクチャ）:
1. MCP `export()` レスポンス: 約100バイト（25トークン）
2. 要約JSON: 約400バイト（100トークン）
3. 合計: 約500バイト（125トークン）

**トークン削減率: 95.8%** 🎉

## まとめ

このノートブックでは、DuckDB × MCP × LLM アーキテクチャを使用して：

1. ✅ LLMのコンテキストを保護（ハンドルのみ返却）
2. ✅ Pythonで効率的にデータ処理（リサンプリング・ローリング平均）
3. ✅ 要約データのみをLLMに返却（1KB制限）
4. ✅ 95%以上のトークン削減を達成

このパターンは、他の時系列分析にも応用可能です。